In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

/kaggle/input/english-to-russian-language/_about.txt
/kaggle/input/english-to-russian-language/rus.txt


In [2]:
english_texts=[]
russian_texts=[]
english_character=[]
russian_character=[]
with open("/kaggle/input/english-to-russian-language/rus.txt", "r", encoding="utf-8") as f:
    lines = f.read().split("\n")
for line in lines[:30000]:
    english_text,russian_text,_=line.split("\t")
    english_texts.append(english_text)
    russian_text = "\t" + russian_text + "\n"
    russian_texts.append(russian_text)

In [3]:
for i in english_texts:
    for c in i:
        if c not in english_character:
            english_character.append(c)
            english_character.sort()

In [4]:
for j in russian_texts:
    for c in j:
        if c not in russian_character:
            russian_character.append(c)
            russian_character.sort()

In [5]:
english_d={}
for i in range(len(english_character)):
    english_d[english_character[i]]=i

In [6]:
russian_d={}
for i in range(len(russian_character)):
    russian_d[russian_character[i]]=i

In [7]:
english_encoder_tokens = len(english_character)
russian_decoder_tokens = len(russian_character)

In [8]:
max_encoder_seq_length=0
for i in english_texts:
    if len(i)>max_encoder_seq_length:
        max_encoder_seq_length=len(i)

In [9]:
max_decoder_seq_length=0
for i in russian_texts:
    if len(i)>max_decoder_seq_length:
        max_decoder_seq_length=len(i)

In [10]:
max_decoder_seq_length

60

In [11]:
encoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(english_texts[bb])
    m=0
    while m<k:
        for char in english_texts[bb][m]:
                           for i in range(len(english_character)):
                                            if english_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    encoder_input_data.append(c)

In [12]:
encoder_input_data=np.array(encoder_input_data)

In [13]:
encoder_input_data.shape

(30000, 17, 74)

In [14]:
decoder_input_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=0
    while m<k:
        for char in russian_texts[bb][m]:
                           for i in range(len(russian_character)):
                                            if russian_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_input_data.append(c)

In [15]:
decoder_input_data=np.array(decoder_input_data)


In [16]:
decoder_input_data.shape


(30000, 60, 101)

In [17]:
decoder_target_data=[]
for bb in range(30000):
    a=[]
    b=[]
    c=[]
    k=len(russian_texts[bb])
    m=1
    while m<k:
        for char in russian_texts[bb][m]:
                           for i in range(len(russian_character)):
                                            if russian_d[char]==i:
                                                a.append(1)
                                            else:
                                                a.append(0)
       
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    m=m-1
    while m<max_decoder_seq_length:
        for i in range(len(russian_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
    decoder_target_data.append(c)

In [18]:
decoder_target_data=np.array(decoder_target_data)


In [19]:
decoder_target_data.shape

(30000, 60, 101)

In [20]:
batch_size = 64  # Batch size for training.
epochs = 100 # Number of epochs to train for.
latent_dim = 256 

In [21]:
from keras.layers import Dense
from keras.layers import LSTM,Input
from keras.models import Model

In [22]:
encoder_inputs = Input(shape=(None,len(english_character)))
encoder = LSTM(latent_dim,dropout=0.2,return_sequences=True,return_state=True)
encoder_outputs_1, state_h_1, state_c_1 = encoder(encoder_inputs)
encoder = LSTM(latent_dim,dropout=0.2,return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_outputs_1)
encoder_states = [state_h_1,state_c_1,state_h, state_c]

In [23]:
decoder_inputs = Input(shape=(None, len(russian_character)))
decoder_lstm = LSTM(latent_dim,return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs_1, _, _ = decoder_lstm(decoder_inputs, initial_state=[state_h_1,state_c_1])
decoder_lstm_1 = LSTM(latent_dim, return_sequences=True,dropout=0.2,return_state=True)
decoder_outputs, _, _ = decoder_lstm_1(decoder_outputs_1, initial_state=[state_h,state_c])
decoder_dense = Dense(len(russian_character), activation="softmax")
decoder_outputs = decoder_dense(decoder_outputs)
model=Model([encoder_inputs, decoder_inputs], decoder_outputs)


In [24]:
model.summary()

Model: "functional_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, None, 74)]   0                                            
__________________________________________________________________________________________________
input_2 (InputLayer)            [(None, None, 101)]  0                                            
__________________________________________________________________________________________________
lstm (LSTM)                     [(None, None, 256),  338944      input_1[0][0]                    
__________________________________________________________________________________________________
lstm_2 (LSTM)                   [(None, None, 256),  366592      input_2[0][0]                    
                                                                 lstm[0][1]            

In [25]:
model.layers[2]

In [26]:
model.compile(
    optimizer="rmsprop", loss="categorical_crossentropy", metrics=["accuracy"]
)
model.fit(
    [encoder_input_data, decoder_input_data],
    decoder_target_data,
    batch_size=batch_size,
    epochs=epochs,
    validation_split=0.2,
)

Epoch 1/100
375/375 [==============================] - 12s 33ms/step - loss: 0.8172 - accuracy: 0.7837 - val_loss: 0.6825 - val_accuracy: 0.8029
Epoch 2/100
375/375 [==============================] - 10s 27ms/step - loss: 0.5882 - accuracy: 0.8315 - val_loss: 0.5796 - val_accuracy: 0.8319
Epoch 3/100
375/375 [==============================] - 10s 27ms/step - loss: 0.5253 - accuracy: 0.8485 - val_loss: 0.5197 - val_accuracy: 0.8490
Epoch 4/100
375/375 [==============================] - 10s 26ms/step - loss: 0.4839 - accuracy: 0.8601 - val_loss: 0.4805 - val_accuracy: 0.8603
Epoch 5/100
375/375 [==============================] - 10s 26ms/step - loss: 0.4533 - accuracy: 0.8687 - val_loss: 0.4474 - val_accuracy: 0.8696
Epoch 6/100
375/375 [==============================] - 10s 28ms/step - loss: 0.4292 - accuracy: 0.8755 - val_loss: 0.4327 - val_accuracy: 0.8735
Epoch 7/100
375/375 [==============================] - 10s 27ms/step - loss: 0.4092 - accuracy: 0.8808 - val_loss: 0.4103 - val_ac

In [27]:
model.save("engtorussian.h5")

In [28]:
import keras

In [29]:
model = keras.models.load_model("engtorussian.h5")
encoder_inputs = model.input[0]  # input_1
encoder_outputs_1, state_h_enc_1, state_c_enc_1 = model.layers[2].output 
encoder_outputs, state_h_enc, state_c_enc = model.layers[4].output 
encoder_states = [state_h_enc_1, state_c_enc_1,state_h_enc, state_c_enc]
encoder_model_1 = keras.Model(encoder_inputs, encoder_states)

In [30]:
decoder_inputs = model.input[1]  
decoder_state_input_h = keras.Input(shape=(latent_dim,),)
decoder_state_input_c = keras.Input(shape=(latent_dim,),)
decoder_state_input_h1 = Input(shape=(latent_dim,),)
decoder_state_input_c1 = Input(shape=(latent_dim,),)
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c,decoder_state_input_h1,decoder_state_input_c1]
decoder_lstm = model.layers[3]
dec_o, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs[:2])
decoder_lstm_1=model.layers[5]
dec_o_1, state_h1, state_c1 = decoder_lstm_1(
    dec_o, initial_state=decoder_states_inputs[-2:])
decoder_states = [state_h,state_c,state_h1,state_c1]
decoder_dense = model.layers[6]
decoder_outputs = decoder_dense(dec_o_1)
decoder_model = keras.Model(
    [decoder_inputs] + decoder_states_inputs, [decoder_outputs] + decoder_states
)

In [31]:
reverse_input_char_index ={}
for i in range(len(english_character)):
    reverse_input_char_index[i]=english_character[i]
reverse_target_char_index ={}
for i in range(len(russian_character)):
    reverse_target_char_index[i]=russian_character[i]

In [32]:
def decode_sequence(input_seq):
    states_value=encoder_model_1.predict(input_seq)
    target_seq = np.zeros((1, 1, len(russian_character)))
    target_seq[0, 0, russian_d["\t"]] = 1.0
    flag=0
    sent=""
    while flag==0:
        output_tokens, h, c,h1,c1 = decoder_model.predict([target_seq] + states_value)
        sample = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sample]
        sent+=sampled_char
        if sampled_char == "\n" or len(sent) > max_decoder_seq_length:
            flag=1
        target_seq = np.zeros((1, 1, len(russian_character)))
        target_seq[0, 0,sample] = 1.0
        states_value = [h, c,h1,c1]
    return sent

In [33]:
english='who?'

In [34]:
k=len(english)
m=0
a=[]
b=[]
c=[]
inpu=[]
while m<k:
    for char in english[m]:
        for i in range(len(english_character)):
            if english_d[char]==i:
                a.append(1)
            else:
                a.append(0)
    for kp in a:
        b.append(kp)
    c.append(b)
    b=[]
    a=[]
    m=m+1
while m<max_encoder_seq_length:
        for i in range(len(english_character)):
            if i==0:
                a.append(1)
            else:
                a.append(0)
        for kp in a:
            b.append(kp)
        c.append(b)
        b=[]
        a=[]
        m=m+1
inpu.append(c)

In [35]:
inpu=np.array(inpu)

In [36]:
inpu

array([[[0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        [0, 0, 0, ..., 0, 0, 0],
        ...,
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0],
        [1, 0, 0, ..., 0, 0, 0]]])

In [37]:
d=decode_sequence(inpu)

In [38]:
print(d)

Кто?

